## Verwendete Biblotheken 

In [ ]:
import couchdb
import json 
import os 
import time 
import kagglehub
import tempfile
import shutil
from pathlib import Path
from dotenv import load_dotenv


Zweck: Den "Gesprächskanal" zwischen Python-Code und dem CouchDB-Server im Docker-Container herstellen und sicherstellen, dass die Zieldatenbank existiert.

In [ ]:
# --- VERBINDUNGS-PARAMETER ---
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984" 
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try:
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich mit CouchDB Server verbunden: {server}")

    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank '{DB_NAME}' existiert bereits.")
    else:
        db = server.create(DB_NAME)
        print(f"Datenbank '{DB_NAME}' wurde neu erstellt.")

except Exception as e:
    print(f"FEHLER bei der Verbindung oder Datenbankerstellung: {e}")
    raise e 

### Arbeitsverzeichnis Pfadanpassung

In [ ]:
base_dir = Path.cwd()
if base_dir.name == "notebooks":
    base_dir = base_dir.parent
    os.chdir(base_dir)
    print(f"Arbeitsverzeichnis ist bereits korrekt: {base_dir}")


### Download


In [ ]:
# --- KONFIGURATION ---
KAGGLE_DATASET = "lucafrance/the-world-factbook-by-cia"
EXPECTED_DOC_COUNT = 258 
data_dir = base_dir / "data"
kaggle_cache_path = Path.home() / ".cache" / "kagglehub" / "datasets" / "lucafrance"

print("--- Überprüfen des Datenbankstatus ---")
try:
    # Zuerst prüfen, ob die Daten schon in der Datenbank sind.
    existing_docs = len(db) # Annahme: 'db' Objekt existiert
    if existing_docs >= EXPECTED_DOC_COUNT:
        print(f"Datenbank '{DB_NAME}' enthält bereits {existing_docs} Dokumente. Import wird übersprungen.")
    else:
        print(f"Datenbank enthält nur {existing_docs} Dokumente. Starte Datenmigration von Kaggle...")

        # Das lokale data-Verzeichnis sicherstellen
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
            print(f"Verzeichnis '{data_dir}' wurde erstellt.")

        # Prüfen, ob die Datei bereits lokal existiert
        local_json_path = data_dir / "countries.json"
        if os.path.exists(local_json_path):
            print(f"Datei existiert bereits lokal unter: {local_json_path}")
            json_path = local_json_path
        else:
            #! --- Download von Kaggle mit kagglehub ---
            print(f"\n--- Lade Datensatz '{KAGGLE_DATASET}' von Kaggle herunter...")
            
            # kagglehub lädt herunter UND entpackt automatisch in das angegebene Verzeichnis
            path = kagglehub.dataset_download(KAGGLE_DATASET)
            print(f"Datensatz erfolgreich nach '{path}' heruntergeladen und entpackt.")
                       
            # Suche nach der JSON-Datei
            json_files = list(Path(path).glob('**/*.json'))
            
            # Kopiere die Datei in das permanente data-Verzeichnis
            source_file = json_files[0]
            shutil.copy2(source_file, local_json_path)
            print(f"Datei '{source_file.name}' wurde nach '{local_json_path}' kopiert.")
            
            json_path = local_json_path    
except Exception as e:
    print(f"\nEin Fehler ist aufgetreten: {e}")
    raise e

finally:
    # --- Schritt 5: Aufräumen ---  
     # Der kagglehub-Cache löschen
    if os.path.exists(kaggle_cache_path):
        try:
            shutil.rmtree(kaggle_cache_path)
            print(f"kagglehub-Cache unter '{kaggle_cache_path}' wurde gelöscht.")
        except Exception as e:
            print(f"WARNUNG: Konnte den kagglehub-Cache nicht löschen: {e}")

In [ ]:
 # --- Verarbeitung der Datei ---
print(f"Verarbeitung der folgenden JSON-Datei: '{json_path}'")

with open(json_path, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

if isinstance(loaded_data, dict):
    countries_data = list(loaded_data.values())
else: 
    countries_data = loaded_data

print(f"{len(countries_data)} Länderobjekte für den Import vorbereitet.")


In [ ]:
# --- Laden der Daten in CouchDB ---

docs_uploaded = len(db)
if docs_uploaded >= EXPECTED_DOC_COUNT:
    print(f"Die Datenbank '{DB_NAME}' enthält bereits {docs_uploaded} Dokumente. Import wird übersprungen.")
else:    
    print(f"\n--- Speichere {len(countries_data)} Dokumente in CouchDB...")
    start_time = time.time()
    for country_doc in countries_data:
        country_doc['type'] = 'country'
        db.save(country_doc)
    end_time = time.time()
    print(f"Alle Dokumente erfolgreich in {end_time - start_time:.2f}s gespeichert.")
